In [1]:
import re
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
##local_path = "/your/path/here"
local_path = "/users/thudson/Documents/Dropbox/chicago-police-data/02-column_std"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/output')

['incomplete',
 'TRRs',
 'complaints-cpd-2016-dec',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-nov',
 'complaints-cpd-2016-oct',
 'complaints-ipra-2016-apr',
 'complaints-merged-2015',
 'shootings-cpd-feb2016',
 'shootings-ipra-may2016']

In [5]:
in_path = path + '/output/complaints-merged-2015/'
out_path = local_path + '/output/complaints-merged-2015/'

In [6]:
try:
    os.mkdir(out_path)
except:
    print("path exists: {}".format(out_path))

path exists: /users/thudson/Documents/Dropbox/chicago-police-data/02-column_std/output/complaints-merged-2015/


In [7]:
##doc_path = '/Data/chicago-police-data/02-column_std'
##column_dict = db.download_file(doc_path+'/doc/','Column_Dictionary.csv')

doc_path = local_path
column_dict = pd.read_csv(doc_path+'/doc/'+'Column_Dictionary.csv').iloc[:,0:3]
column_dict.head()

,Orig Column Name,New Name,Alternative Name
0,Victim_Age,Civilian.Victim.Age,NaN
1,Victim_Gender,Civilian.Victim.Gender,NaN
2,Victim_Race,Civilian.Victim.Race_Code,NaN
3,Witness_Age,Civilian.Witness.Age,NaN
4,Witness_Gender,Civilian.Witness.Gender,NaN


In [8]:
## SOMETHING TO PUT IN UTILS? HOW WOULD THE SPECIFIED COLUMN_DICT WORK?

def column_mapper(df,
                  column_dict = column_dict):
    current_columns = pd.DataFrame(df.columns.tolist(),columns=['Orig Column Name'])
    new_columns = current_columns.merge(column_dict,how='left',on='Orig Column Name')['New Name'].tolist() 
    if current_columns.shape[0] == len(new_columns):
        return new_columns
    else:
        print("Join failed, more columns returned than expected")

## August 2015 Data

### Report 1

In [9]:
in_path_aug_2015_report = in_path+'august_2015/'
out_path_aug_2015_report = out_path

files = db.list_files(in_path_aug_2015_report)
files

['foia_14-5509_-_birth_year_of_accused.csv',
 'foia_14-5509_-_birth_year_of_accused.xlsx',
 'foia_14-5509_-_birth_year_of_accused_metadata.csv',
 'foia_14-5509_-_complaining_witness_data.csv',
 'foia_14-5509_-_complaining_witness_data.xlsx',
 'foia_14-5509_-_complaining_witness_data_metadata.csv',
 'foia_14-5509_-_investigator_data.csv',
 'foia_14-5509_-_investigator_data.xlsx',
 'foia_14-5509_-_investigator_data_metadata.csv']

In [10]:
file = files[0]
df = db.download_file(in_path_aug_2015_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-merged-2015/august_2015/foia_14-5509_-_birth_year_of_accused.csv


['Accused_Full_Name',
 'Year_of_Birth',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [11]:
df.columns = column_mapper(df)
df.columns.tolist()

['Officer.Accused.Full_Name',
 'Officer.Accused.Birth_Year',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [12]:
df.to_csv(out_path_aug_2015_report+'aug_2015'+file,index=False)

### Report 2

In [13]:
file = files[3]
df = db.download_file(in_path_aug_2015_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-merged-2015/august_2015/foia_14-5509_-_complaining_witness_data.csv


['CRID',
 'Witness_Gender',
 'Witness_Age',
 'Witness_Race',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [14]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Civilian.Witness.Gender',
 'Civilian.Witness.Age',
 'Civilian.Witness.Race_Code',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [15]:
df.to_csv(out_path_aug_2015_report+'aug_2015'+file,index=False)

### Report 3

In [16]:
file = files[6]
df = db.download_file(in_path_aug_2015_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-merged-2015/august_2015/foia_14-5509_-_investigator_data.csv


['Investigator_Last_Name',
 'Investigator_First_Name',
 'Current_Report',
 'Current_Rank',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [17]:
df.columns.values[3]='Investigator_Current_Rank'
df.columns = column_mapper(df)


['Investigator.Last_Name',
 'Investigator.First_Name',
 'Investigator.Current_Report',
 'Officer.Accused.Rank',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [18]:
df.columns.tolist()

['Investigator.Last_Name',
 'Investigator.First_Name',
 'Investigator.Current_Report',
 'Investigator.Rank',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [19]:
df.to_csv(out_path_aug_2015_report+'aug_2015'+file,index=False)